In [9]:
# The code was removed by Watson Studio for sharing.

# IBM Applied Data Science Capstone Course by Coursera

### Final Code:  Week 5

## Impact of Parks on the pollution levels of Delhi, India

## -----------------------------------------------------------------------------

## 1. Importing Libraries

In [2]:
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    python_abi:    

In [3]:

import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
#!conda install -c conda-forge geocoder --yes
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
print("Libraries imported.")

Libraries imported.


## 2. Scrap data from Wikipedia page into a DataFrame

In [10]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# creating a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("li"):
    neighborhoodList.append(row.text)
#The list contains first 12 non required names.Cleaning those names for best results.
del neighborhoodList[0:12]
del neighborhoodList[187:505]
    # create a new DataFrame from the list
dl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

dl_df.head()

,Neighborhood
0,Adarsh Nagar
1,Ashok Vihar
2,Azadpur
3,Bawana
4,Begum Pur


In [11]:
dl_df.shape

(187, 1)

## 3. Get the geographical coordinates

In [12]:
import geocoder
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, New Delhi,India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# Using function to get the coordinates.
coords = [ get_latlng(neighborhood) for neighborhood in dl_df["Neighborhood"].tolist() ]

In [14]:
coords

[[28.720350000000053, 77.17264000000006],
 [28.69037000000003, 77.17609000000004],
 [28.712420000000066, 77.17304000000007],
 [28.79767000000004, 77.04522000000003],
 [28.732620000000054, 77.05213000000003],
 [28.707092837280925, 77.20451789081991],
 [28.728770000000054, 77.17557000000005],
 [28.737330000000043, 77.03959000000003],
 [28.688010000000077, 77.15866000000005],
 [28.711690000000033, 77.20197000000007],
 [28.69659000000007, 77.13892000000004],
 [28.705010000000073, 77.18950000000007],
 [28.83979000000005, 77.07696000000004],
 [28.69590000000005, 77.13725000000005],
 [28.68584000000004, 77.13188000000008],
 [28.719470000000058, 77.10147000000006],
 [28.733560000000068, 77.10401000000007],
 [28.714230000000043, 77.15744000000007],
 [28.67557000000005, 77.16190000000006],
 [28.660990000000027, 77.20450000000005],
 [28.656270000000063, 77.23232000000007],
 [28.648580000000038, 77.23071000000004],
 [28.67671000000007, 77.21767000000006],
 [28.65457890544559, 77.23339989939495],
 

In [15]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the original dataframe
dl_df['Latitude'] = df_coords['Latitude']
dl_df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(dl_df.shape)
dl_df

(187, 3)


,Neighborhood,Latitude,Longitude
0,Adarsh Nagar,28.720350,77.172640
1,Ashok Vihar,28.690370,77.176090
2,Azadpur,28.712420,77.173040
3,Bawana,28.797670,77.045220
4,Begum Pur,28.732620,77.052130
5,Dhaka,28.707093,77.204518
6,Jahangirpuri,28.728770,77.175570
7,Karala,28.737330,77.039590
8,Keshav Puram,28.688010,77.158660
9,Kingsway Camp,28.711690,77.201970


In [26]:
# save the DataFrame as CSV file
project.save_data(file_name = "dl_df.csv",data = dl_df.to_csv(index=False))

{'file_name': 'dl_df.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'mydatascienceproject-donotdelete-pr-kig7zidyusboov',
 'asset_id': '90b23586-842b-4ecb-a305-ab146c2f93de'}

## 4. Create a map of Delhi with neighborhoods superimposed on top

In [18]:
# get the coordinates of Delhi
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi, India are :  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi, India are :  28.6141793, 77.2022662.


In [19]:
#create map of Delhi using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [32]:
# save the map as HTML file
map_kl.save('map_dl.html')


In [20]:
# The code was removed by Watson Studio for sharing.

Credentials Applied Successfully


## Define Foursquare Credentials and Version
CLIENT_ID = # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = # Foursquare API version


## 5. Use the Foursquare API to explore the neighborhoods

In [21]:
#Now, let's get the top 100 venues that are within a radius of 2000 meters.

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dl_df['Latitude'], dl_df['Longitude'], dl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# New Venues DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7312, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72035,77.17264,Ever Bake,28.706419,77.162607,Bakery
1,Adarsh Nagar,28.72035,77.17264,Shalimar bagh club,28.711882,77.157933,Indian Restaurant
2,Adarsh Nagar,28.72035,77.17264,Arjun Pao Bhaji,28.708468,77.184963,Indian Restaurant
3,Adarsh Nagar,28.72035,77.17264,Domino's Pizza,28.706000,77.163000,Pizza Place
4,Adarsh Nagar,28.72035,77.17264,Cafe Coffee Day,28.707934,77.161478,Coffee Shop


In [24]:
# save the DataFrame as CSV file
project.save_data(file_name = "venues_df.csv",data = venues_df.to_csv(index=False), overwrite = True)

{'file_name': 'venues_df.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'mydatascienceproject-donotdelete-pr-kig7zidyusboov',
 'asset_id': '071fb2d1-df0a-414f-866f-d6de22f7bbb1'}

## 6.Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 222 uniques categories.


In [26]:
# Checking the list of categories
venues_df['VenueCategory'].unique()[:222]

array(['Bakery', 'Indian Restaurant', 'Pizza Place', 'Coffee Shop',
       'Park', 'Bus Station', 'Golf Course', 'Hotel',
       'Athletics & Sports', 'Asian Restaurant', 'Sandwich Place',
       'Snack Place', 'Department Store', 'South Indian Restaurant',
       'Fast Food Restaurant', 'Market', 'Dessert Shop',
       'Basketball Court', 'Train Station', 'Light Rail Station',
       'Restaurant', 'Multiplex', 'Donut Shop', 'Convenience Store',
       'Gym / Fitness Center', 'Shopping Mall', 'Frozen Yogurt Shop',
       'ATM', 'Campground', 'Playground', 'Accessories Store',
       'Business Service', 'Café', 'Chinese Restaurant', 'Food Truck',
       'Burger Joint', 'Lake', 'Track Stadium', 'Tea Room',
       'Electronics Store', 'History Museum', 'Pool Hall', 'Diner',
       'Arts & Crafts Store', 'Clothing Store', 'Building', 'Airport',
       'Theme Park', 'Vegetarian / Vegan Restaurant', "Women's Store",
       'Food & Drink Shop', 'BBQ Joint', 'Miscellaneous Shop', 'Motel',
    

In [27]:
# check if the results contain "Park"
"Park" in venues_df['VenueCategory'].unique()

True

In [42]:
# check how many "Park" do we have
venues_df['VenueCategory'].count()
venues_df.loc[venues_df.VenueCategory == 'Park', 'VenueCategory'].count()

101

## 7. Analyzing

In [29]:
# one hot encoding
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(7312, 223)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
#Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(176, 223)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Resort,Restaurant,River,Road,Salad Place,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [41]:
len(dl_grouped[dl_grouped["Park"] > 0])

58

### Create a new DataFrame for Parks data only

In [43]:
dl_park = dl_grouped[["Neighborhoods","Park"]]

In [44]:
dl_park.head()

,Neighborhoods,Park
0,Adarsh Nagar,0.2
1,Alaknanda,0.0
2,Anand Vihar,0.0
3,Ashok Nagar,0.5
4,Ashok Vihar,0.0


## 8. Cluster Neighbourhoods¶ 

Working with 5 Clusters.

In [61]:
kclusters = 5

dl_clustering = dl_park.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 1, 2, 2, 2, 2, 4, 4], dtype=int32)

In [62]:
# create a new dataframe that includes the cluster as well as the Park.
dl_merged = dl_park.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

In [63]:
dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Park,Cluster Labels
0,Adarsh Nagar,0.2,3
1,Alaknanda,0.0,2
2,Anand Vihar,0.0,2
3,Ashok Nagar,0.5,1
4,Ashok Vihar,0.0,2


In [64]:
# merge grouped data latitude/longitude for each neighborhood
dl_merged = dl_merged.join(dl_df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head()

(186, 5)


,Neighborhood,Park,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.2,3,28.72035,77.17264
1,Alaknanda,0.0,2,28.52567,77.25325
2,Anand Vihar,0.0,2,28.69257,77.12489
3,Ashok Nagar,0.5,1,28.69223,77.30124
4,Ashok Vihar,0.0,2,28.69037,77.17609


In [65]:
# sort the results by Cluster Labels
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(186, 5)


,Neighborhood,Park,Cluster Labels,Latitude,Longitude
89,Mayur Vihar,0.058824,0,28.612795,77.288501
173,Vivek Vihar,0.133333,0,28.668730,77.318950
172,Vishwas Nagar,0.058824,0,28.665120,77.294310
152,Shastri Nagar,0.076923,0,28.674350,77.179270
152,Shastri Nagar,0.076923,0,28.674350,77.179270
149,Shakarpur,0.066667,0,28.627150,77.279400
143,Sarai Rohilla,0.076923,0,28.665830,77.184810
132,Rohini Sub City,0.111111,0,28.733560,77.104010
131,Rithala,0.090909,0,28.719470,77.101470
113,Pandav Nagar,0.071429,0,28.614580,77.275740


In [66]:
# Lets Check them on Map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 9. Viewing the Clusters

In [67]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
89,Mayur Vihar,0.058824,0,28.612795,77.288501
173,Vivek Vihar,0.133333,0,28.668730,77.318950
172,Vishwas Nagar,0.058824,0,28.665120,77.294310
152,Shastri Nagar,0.076923,0,28.674350,77.179270
152,Shastri Nagar,0.076923,0,28.674350,77.179270
149,Shakarpur,0.066667,0,28.627150,77.279400
143,Sarai Rohilla,0.076923,0,28.665830,77.184810
132,Rohini Sub City,0.111111,0,28.733560,77.104010
131,Rithala,0.090909,0,28.719470,77.101470
113,Pandav Nagar,0.071429,0,28.614580,77.275740


In [68]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
88,Maujpur,0.4,1,28.689575,77.278262
147,Shahdara,0.4,1,28.689570,77.278020
147,Shahdara,0.4,1,28.689570,77.278020
3,Ashok Nagar,0.5,1,28.692230,77.301240


In [69]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
13,Brij Puri,0.00,2,28.705140,77.272440
114,Partap Nagar,0.00,2,28.669770,77.195750
115,Paschim Vihar,0.00,2,28.669330,77.091730
116,Patel Nagar,0.00,2,28.647830,77.164490
117,Patparganj,0.00,2,28.629480,77.307660
118,Pitam Pura,0.00,2,28.695900,77.137250
120,Pratap Nagar,0.00,2,28.669770,77.195750
124,Punjabi Bagh,0.00,2,28.666340,77.125000
125,Rajender Nagar,0.00,2,28.633509,77.173864
126,Rajokri,0.00,2,28.519370,77.121260


In [70]:
dl_merged.loc[dl_merged['Cluster Labels'] == 3]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
155,Silampur,0.20,3,28.67011,77.27122
0,Adarsh Nagar,0.20,3,28.72035,77.17264
101,Naveen Shahdara,0.25,3,28.67369,77.28326


In [71]:
dl_merged.loc[dl_merged['Cluster Labels'] == 4]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
158,South Extension,0.034091,4,28.562490,77.215170
109,Nizamuddin West,0.020408,4,28.588650,77.244860
160,Sundar Nagar[1],0.030000,4,28.603520,77.243020
68,Khan Market,0.040000,4,28.600450,77.225980
106,Netaji Nagar,0.017241,4,28.577460,77.185170
102,Neeti Bagh,0.030000,4,28.559560,77.218382
164,Tis Hazari,0.020833,4,28.666000,77.215200
9,Bara Hindu Rao,0.028571,4,28.660990,77.204500
44,Green Park,0.030000,4,28.558970,77.204620
76,Kotla Mubarakpur,0.022472,4,28.574350,77.224190


## 10.Observations

As we can see there are Good amount of parks in Cluster with Label 1 which is reflected as Violet Color on Map. With Frequencies between 0.4 to 0.5. This is the best Cluster. Then we have Cluster with Label 3 which is Light Green colored. It shows 2nd best areas in term of Parks. Then we have Clusters with Label 0 with Red Colored Dots. These are Average level Parks frequencies that one area should have.And then Cluster Label 4 with Skin Color dots lies below average with frequencies as 0.01 to 0.04.

At last Cluster with Label 2 colored as Light blue which consists major neighborhood areas are at the bottom of the Parks Frequencies list which shows values between 0.00 and 0.01. With maximum 0.00 values it shows why Delhi is one of the highest polluted cities in the world. Observations Clearly states that Maximum regions in delhi are a below average in rating for Parks. Having maintained an average level of parks in all regions would never give rise to High pollution atmosphere in places like capital of India

# The End 

# Thank you!